# SQL Cleaning Maps Data for Final Project

####Import key libraries and data file with information to connect to MySQL

In [304]:
import numpy as np
import pandas as pd
import MySQLdb as mdb
import MySQL_data_file as MySQL_data

####Create connection object "con" for accessing databases:

In [305]:
con = mdb.connect(MySQL_data.my_sql_host, MySQL_data.my_sql_user,\
                    MySQL_data.my_sql_passwd,\
                    MySQL_data.my_sql_database)
#con.close

####Define key functions for querying information from MySQL databases:

In [306]:
def table_information(table_name, columns_only=False):
    """
    Function returns table information. Will return list of columns if columns_only is set
    """
    cur = con.cursor()
    cur.execute("DESCRIBE {}".format(table_name))
    if columns_only == False:
        return [x for x in cur.fetchall()]
    else:
        #column_list = []
        return [row[0:1][0] for row in cur.fetchall()]      
#table_information('mmaps_data', True)

In [307]:
def run_sql(query):
    cur = con.cursor()
    cur.execute(query)
    con.commit()
    #con.close()
    return cur.fetchall()

####Initial Test Queries

In [176]:
#run_sql("""SHOW TABLES""")
#run_sql("""SELECT datetime,origins,destinations,duration FROM gmaps_data""")
#table_count = run_sql("""SELECT datetime, FROM gmaps_data limit 1""")

####Adjusting Date and Time Formats in Python

In [321]:
#Converting datetime to ISO format in Python
sample_dt_object = run_sql("""SELECT datetime FROM gmaps_data limit 1""")
sample_dt_object_iso = sample_dt_object[0][0].isoformat(" ")
#sample_dt_object_iso

####Adjusting Date and Time Formats in SQL

In [323]:
#Working with timestamps and time functions in SQL
#run_sql("""SELECT CURRENT_TIMESTAMP""")#or NOW() or CURRENT_TIMESTAMP()
#run_sql("""SELECT HOUR(datetime) FROM gmaps_data limit 10""")
#run_sql("""SELECT UNIX_TIMESTAMP(datetime) FROM gmaps_data limit 10""")
#run_sql("""SELECT GET_FORMAT(datetime, 'ISO') FROM gmaps_data limit 1""")

In [325]:
#retrieving time data in ISO format (through SQL)
#run_sql("""SELECT DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s') FROM gmaps_data limit 3""")
#converting to PST time zone
#run_sql("""SELECT CONVERT_TZ(DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s'),'+00:00','-07:00') FROM gmaps_data limit 1""")

In [120]:
run_sql("""SELECT DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s'),origins,destinations,duration_traffic 
        FROM mmaps_data
        WHERE datetime BETWEEN '2015-05-02 11:00:00' AND '2015-05-02 11:14:00'
        AND origins LIKE 'Pac%'
        LIMIT 1
        """)

(('2015-05-02 11:00:02',
  'Pacific Heights, San Francisco, CA',
  'Oakland, CA',
  2005L),)

In [129]:
#The code below shows transit data between 2am and 11am on Saturday May 2 
#traveling from Pac Heights to Financial District
len(run_sql("""SELECT DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s'),origins,destinations,duration_traffic 
        FROM mmaps_data
        WHERE 
            ((datetime BETWEEN '2015-05-01 11:00:00' AND '2015-05-01 18:01:00')
            OR 
            (datetime BETWEEN '2015-05-01 21:00:00' AND '2015-05-02 04:01:00'))
        AND 
            ((origins LIKE 'Pac%' AND destinations LIKE 'Fin%')
            OR 
            (origins LIKE 'Fin%'and destinations LIKE 'Pac%'))
        ORDER BY datetime
        """))

58

In [168]:
#different style but same result as previous SQL statement
run_sql("""SELECT DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s'),origins,destinations,duration_traffic 
        FROM mmaps_data
        WHERE 
        ((datetime BETWEEN '2015-05-01 11:00:00' AND '2015-05-01 18:01:00')
        AND (origins LIKE 'Pac%') AND (destinations LIKE 'Fin%')) 
        OR
        ((datetime BETWEEN '2015-05-01 21:00:00' AND '2015-05-02 04:01:00')
        AND (origins LIKE 'Fin%') AND (destinations LIKE 'Pac%')) 
        ORDER BY datetime
        LIMIT 1
        """)

(('2015-05-01 11:00:03',
  'Pacific Heights, San Francisco, CA',
  'Financial District, San Francisco, CA',
  767L),)

In [191]:
#different style but same result as previous SQL statement
#all of the data is sorted in the ideal way (seconds, minutes, hours, dates)
#so we can see the duration values for the same time over multiple days
#next to each other with respect to the rows
len(run_sql("""SELECT DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s'),origins,destinations,duration_traffic 
        FROM mmaps_data
        WHERE 
        ((TIME(datetime) BETWEEN '11:00:00' AND '18:01:00')
        AND (origins LIKE 'Pac%') AND (destinations LIKE 'Fin%')) 
        OR
        ((TIME(datetime) BETWEEN '21:00:00' AND '24:00:00' OR
        (TIME(datetime) BETWEEN '00:00:00' AND '04:01:00'))
        AND (origins LIKE 'Fin%') AND (destinations LIKE 'Pac%'))
        ORDER BY HOUR(datetime), MINUTE(datetime), DATE(datetime)
        """))

327

In [9]:
#Took the code from above and specified the query such that we are looking
#at the morning driving times for the commute from pac heights to financial district
run_sql("""SELECT DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s'),origins,destinations,duration_traffic
        FROM mmaps_data
        WHERE 
        ((TIME(datetime) BETWEEN '11:00:00' AND '18:01:00')
        AND (origins LIKE 'Pac%') AND (destinations LIKE 'Fin%'))
        ORDER BY HOUR(datetime), MINUTE(datetime), DATE(datetime)
        LIMIT 1
        """)

(('2015-04-27 11:00:03',
  'Pacific Heights, San Francisco, CA',
  'Financial District, San Francisco, CA',
  723L),)

In [14]:
run_sql("""select DISTINCT destinations from gmaps_data""")

(('Financial District, San Francisco, CA',),
 ('Oakland, CA',),
 ('Mountain View, CA',),
 ('Berkeley, CA',),
 ('Outer Sunset, San Francisco, CA',),
 ('Noe Valley, San+Francisco, CA',),
 ('Pacific Heights, San Francisco, CA',),
 ('Outer Richmond, San+Francisco, CA',),
 ('Mission District, San Francisco, CA',),
 ('Russian Hill, San Francisco, CA',),
 ('North Beach, San Francisco, CA',))

In [267]:
#update one of the tables to remove USA
run_sql("""UPDATE gmaps_data
        SET destinations=(SELECT DISTINCT destinations FROM mmaps_data WHERE destinations LIKE 'North%')
        WHERE destinations LIKE 'North%'
        """)

()

In [138]:
#join tables

In [270]:
#Below is an example of combining the two tables with union
#NOTE: this is not preferrable because you cannot ORDER BY datetime without error
len(run_sql("""SELECT DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s'),origins,destinations,duration_traffic,travel_mode
        FROM mmaps_data
        WHERE 
        ((TIME(datetime) BETWEEN '11:00:00' AND '18:01:00')
        AND (origins LIKE 'Pac%') AND (destinations LIKE 'Fin%'))
        UNION ALL
        SELECT DATE_FORMAT(datetime, '%Y-%m-%d %H:%i:%s'),origins,destinations,duration,travel_mode
        FROM gmaps_data
        WHERE 
        ((TIME(datetime) BETWEEN '11:00:00' AND '18:01:00')
        AND (origins LIKE 'Pac%') AND (destinations LIKE 'Fin%'))
        """))

464

In [268]:
#Try An update statement

In [ ]:
#try some sort of inner join

In [52]:
run_sql("""SELECT DATE_FORMAT(datetime,'%H:%i') as datetime, origins, destinations, AVG(duration_traffic) / 60 as MINUTES
        FROM mmaps_data
        WHERE 
        ((TIME(datetime) BETWEEN '11:00:00' AND '18:01:00')
        AND (origins LIKE 'Berk%') AND (destinations LIKE 'Moun%'))
        GROUP BY HOUR(datetime), MINUTE(datetime)
        ORDER BY HOUR(datetime), MINUTE(datetime), DATE(datetime)
        LIMIT 1
        """)

(('11:00', 'Berkeley, CA', 'Mountain View, CA', Decimal('59.83611111')),)

In [100]:
test_df = pd.read_sql("""SELECT DATE_FORMAT(datetime,'%H:%i') as datetime, origins, destinations, AVG(duration_traffic) / 60 as MINUTES
        FROM mmaps_data
        WHERE 
        (TIME(datetime) BETWEEN '11:00:00' AND '18:01:00')
        GROUP BY origins,destinations,HOUR(datetime), MINUTE(datetime)
        ORDER BY HOUR(datetime), MINUTE(datetime),origins,destinations
        """, con)
test_df.head(26)

,datetime,origins,destinations,MINUTES
0,11:00,"Berkeley, CA","Financial District, San Francisco, CA",27.136111
1,11:00,"Berkeley, CA","Mountain View, CA",59.836111
2,11:00,"Berkeley, CA","Oakland, CA",17.683333
3,11:00,"Mission District, San Francisco, CA","Financial District, San Francisco, CA",23.919444
4,11:00,"Mission District, San Francisco, CA","Mountain View, CA",43.208333
5,11:00,"Mission District, San Francisco, CA","Oakland, CA",26.119444
6,11:00,"Noe Valley, San+Francisco, CA","Financial District, San Francisco, CA",25.761111
7,11:00,"Noe Valley, San+Francisco, CA","Mountain View, CA",46.627778
8,11:00,"Noe Valley, San+Francisco, CA","Oakland, CA",30.791667
9,11:00,"North Beach, San Francisco, CA","Financial District, San Francisco, CA",6.444444


####Combinging latest google and bing maps data into dataframes below:

In [279]:
this_df1 = pd.read_sql("""SELECT entry_id,timestamp,datetime,origins,destinations,travel_mode,duration,distance,fare
                        FROM gmaps_data""",con)

In [280]:
this_df2 = pd.read_sql("""SELECT entry_id,timestamp,datetime,origins,destinations,travel_mode,duration_traffic,distance,congestion
                        FROM mmaps_data""",con)

In [296]:
#print this_df1.columns
#print this_df2.columns
this_df2.columns = [u'entry_id', u'timestamp', u'datetime', u'origins', u'destinations', u'travel_mode', u'duration', u'distance', u'congestion']

In [285]:
this_df3 = this_df1.merge(this_df2, how='outer')

In [289]:
combined_df = this_df3.copy()

In [295]:
#combined_df.info()
combined_df.iloc[:5,1:7]

,timestamp,datetime,origins,destinations,travel_mode,duration
0,2015-04-20 01:14:42,2015-04-20 08:14:42,"Outer Sunset, San Francisco, CA, USA","Financial District, San Francisco, CA, USA",transit,2661
1,2015-04-27 02:32:58,2015-04-27 09:32:56,"Oakland, CA","Pacific Heights, San Francisco, CA",driving,1750
2,2015-04-20 01:14:42,2015-04-20 08:14:42,"Outer Sunset, San Francisco, CA, USA","Oakland, CA, USA",transit,6056
3,2015-04-27 02:32:58,2015-04-27 09:32:56,"Oakland, CA","Outer Richmond, San+Francisco, CA",driving,2089
4,2015-04-20 01:14:42,2015-04-20 08:14:42,"Noe Valley, San Francisco, CA, USA","Financial District, San Francisco, CA, USA",transit,3182
